# path and outputs

In [8]:
from pathlib import Path

# --- Project base ---
BASE = Path("/storage/users/job37yv/Projects/Franziska_faber")

# --- Input (QC-prepared counts + metadata) ---
COUNT_INPUT_DIR = BASE / "data" / "processed_data" / "qc_fractional_counts"

# --- Storage for intermediate/cleaned objects (h5ad, merged counts) ---
DATA_PROCESSED_DIR = BASE / "data" / "processed_data"

# --- Analysis outputs (all results and plots) ---
ANALYSIS_DIR = BASE / "analysis"
ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

# --- Subdirectories for downstream analysis ---
QC_DIR   = ANALYSIS_DIR / "QC"
DE_DIR   = ANALYSIS_DIR / "DE"
GSEA_DIR = ANALYSIS_DIR / "GSEA"
ORA_DIR  = ANALYSIS_DIR / "ORA"
FIG_DIR  = ANALYSIS_DIR / "figures"

for d in [QC_DIR, DE_DIR, GSEA_DIR, ORA_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("[OK] Directory structure ready:")
print(" - Input counts:", COUNT_INPUT_DIR)
print(" - Processed data store:", DATA_PROCESSED_DIR)
print(" - Analysis base:", ANALYSIS_DIR)
for d in [QC_DIR, DE_DIR, GSEA_DIR, ORA_DIR, FIG_DIR]:
    print("   -", d)


[OK] Directory structure ready:
 - Input counts: /storage/users/job37yv/Projects/Franziska_faber/data/processed_data/qc_fractional_counts
 - Processed data store: /storage/users/job37yv/Projects/Franziska_faber/data/processed_data
 - Analysis base: /storage/users/job37yv/Projects/Franziska_faber/analysis
   - /storage/users/job37yv/Projects/Franziska_faber/analysis/QC
   - /storage/users/job37yv/Projects/Franziska_faber/analysis/DE
   - /storage/users/job37yv/Projects/Franziska_faber/analysis/GSEA
   - /storage/users/job37yv/Projects/Franziska_faber/analysis/ORA
   - /storage/users/job37yv/Projects/Franziska_faber/analysis/figures


# Load data and build adata object

In [13]:
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project paths (as you specified) ---
BASE = Path("/storage/users/job37yv/Projects/Franziska_faber")
COUNT_INPUT_DIR     = BASE / "data" / "processed_data" / "qc_fractional_counts"  # contains counts_fractional.tsv + sample_metadata.tsv
DATA_PROCESSED_DIR  = BASE / "data" / "processed_data"
ANALYSIS_DIR        = BASE / "analysis"
H5AD                = DATA_PROCESSED_DIR / "host_bulk_fractional_counts.h5ad"

# read TSVs written earlier
counts_path = COUNT_INPUT_DIR / "counts_fractional.tsv"   # samples x genes (raw fractional)
meta_path   = COUNT_INPUT_DIR / "sample_metadata.tsv"     # sample metadata

counts = pd.read_csv(counts_path, sep="\t", index_col=0)
meta   = pd.read_csv(meta_path,   sep="\t", index_col=0)

# align (just in case)
counts = counts.loc[meta.index]

# build AnnData (if available) and add norm/log1p
adata = None
try:
    import anndata as ad
    adata = ad.AnnData(X=counts.values.astype(float))
    adata.obs = meta.copy()
    adata.var_names = counts.columns.astype(str)
    adata.obs_names = counts.index.astype(str)

    # layers
    adata.layers["counts"] = counts.values.astype(float)

    # size-factor normalization (median-of-ratios) for overview
    GxS = adata.layers["counts"].T                     # genes x samples
    with np.errstate(divide='ignore', invalid='ignore'):
        logX = np.log(GxS); logX[~np.isfinite(logX)] = np.nan
    gmeans = np.exp(np.nanmean(logX, axis=1)); gmeans[gmeans==0] = np.nan
    ratios = GxS / gmeans[:, None]
    sfs = np.nanmedian(ratios, axis=0); sfs[~np.isfinite(sfs)] = 1.0
    norm = adata.layers["counts"] / sfs[None, :]
    adata.layers["norm"]  = norm
    adata.layers["log1p"] = np.log1p(norm)

    adata.uns["counts_type"] = "featureCounts fractional (-M --fraction)"
    adata.uns["notes"] = "Built from merged TSVs (qc_fractional_counts). layers: counts/norm/log1p."

    DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
    adata.write_h5ad(H5AD)
    print("[OK] AnnData rebuilt from TSVs and saved to:", H5AD)
    print(adata)
except Exception as e:
    print("[WARN] anndata not available:", e)
    print("You can still proceed with pandas DataFrames `counts` (samples x genes) and `meta`.")


[WARN] anndata not available: operands could not be broadcast together with shapes (12,67665) (1,12) 
You can still proceed with pandas DataFrames `counts` (samples x genes) and `meta`.


/tmp/ipykernel_896322/799461717.py:38: RuntimeWarning: Mean of empty slice
  gmeans = np.exp(np.nanmean(logX, axis=1)); gmeans[gmeans==0] = np.nan


In [ ]:
adata

# map your Geneid → symbols (for GSEA/ORA)

In [37]:
import pandas as pd

def build_ensembl_mappings(adata_ens, species="human"):
    """
    Build dictionaries:
      - ensembl (with version) → entrez (NCBI Gene ID)
      - ensembl (with version) → uniprot (Swiss-Prot accession)
    """
    ids = pd.Index(adata_ens.var_names.astype(str))

    # --- strip version suffix (e.g., ENSG00000141562.13 → ENSG00000141562) ---
    ensembl_core = ids.str.replace(r"\.\d+$", "", regex=True)

    try:
        import mygene
        mg = mygene.MyGeneInfo()

        q = mg.querymany(
            ensembl_core.tolist(),
            scopes="ensembl.gene",
            fields="entrezgene,uniprot.Swiss-Prot",
            species=species,
            as_dataframe=True,
            returnall=False,
            verbose=False
        )

        if not isinstance(q, pd.DataFrame) or q.empty:
            print("[WARN] mygene returned nothing.")
            return {}, {}

        # Clean results
        df = q.reset_index().rename(columns={"query": "ensembl_core"})
        df = df.drop_duplicates("ensembl_core")

        # Build mapping back to *original IDs with version*
        core2ver = pd.Series(ids.values, index=ensembl_core.values)

        ens2entrez = {}
        ens2uniprot = {}
        for _, row in df.iterrows():
            ens_core = row["ensembl_core"]
            ens_ver  = core2ver.get(ens_core)   # restore versioned ID
            if ens_ver is None:
                continue
            ens2entrez[ens_ver]  = row.get("entrezgene")
            ens2uniprot[ens_ver] = row.get("uniprot.Swiss-Prot")

        print(f"[MAP] {len(ens2entrez)} Ensembl IDs mapped to Entrez/UniProt "
              f"(of {len(ids)} input).")
        return ens2entrez, ens2uniprot

    except Exception as e:
        print(f"[WARN] mapping failed ({e}); returning empty dicts.")
        return {}, {}


In [38]:
adata_ens = filter_and_annotate_ensembl(adata, species="human")  # or "mouse"
adata_ens.var.head()

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


[INFO] kept 67016/67665 features (Ensembl only).
[ANN] mapped 50135/67016 features to symbols.


,ensembl_core,gene_symbol
ENSG00000223972.5,ENSG00000223972,DDX11L1
ENSG00000227232.5,ENSG00000227232,WASH7P
ENSG00000278267.1,ENSG00000278267,MIR6859-1
ENSG00000243485.5,ENSG00000243485,MIR1302-2HG
ENSG00000284332.1,ENSG00000284332,MIR1302-2


In [31]:
adata_ens.var['gene_symbol']

ENSG00000223972.5            DDX11L1
ENSG00000227232.5             WASH7P
ENSG00000278267.1          MIR6859-1
ENSG00000243485.5        MIR1302-2HG
ENSG00000284332.1          MIR1302-2
                          ...       
ENSG00000276017.1    ENSG00000276017
ENSG00000278817.1    ENSG00000278817
ENSG00000277196.4    ENSG00000277196
ENSG00000278625.1       LOC124905334
ENSG00000277374.1                 U1
Name: gene_symbol, Length: 67016, dtype: object

## mapping dictionaries

In [33]:
import pandas as pd

def build_ensembl_mappings(adata_ens, species="human"):
    """
    Build dictionaries:
      - ensembl → entrez (NCBI Gene ID)
      - ensembl → uniprot (Swiss-Prot accession)
    """
    ids = pd.Index(adata_ens.var_names.astype(str))

    try:
        import mygene
        mg = mygene.MyGeneInfo()

        q = mg.querymany(
            ids.tolist(),
            scopes="ensembl.gene",
            fields="entrezgene,uniprot.Swiss-Prot",
            species=species,
            as_dataframe=True,
            returnall=False,
            verbose=False
        )

        if not isinstance(q, pd.DataFrame) or q.empty:
            print("[WARN] mygene returned nothing.")
            return {}, {}

        df = q.reset_index().rename(columns={"query": "ensembl"}).drop_duplicates("ensembl")

        # dictionaries
        ens2entrez = dict(zip(df["ensembl"], df.get("entrezgene", [None]*len(df))))
        ens2uniprot = dict(zip(df["ensembl"], df.get("uniprot.Swiss-Prot", [None]*len(df))))

        print(f"[MAP] {len(ens2entrez)} Ensembl IDs mapped to Entrez and UniProt.")
        return ens2entrez, ens2uniprot

    except Exception as e:
        print(f"[WARN] mapping failed ({e}); returning empty dicts.")
        return {}, {}


In [64]:
ens2entrez, ens2uniprot = build_ensembl_mappings(adata_ens, species="human")



Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


[MAP] 67016 Ensembl IDs mapped to Entrez/UniProt (of 67016 input).


In [65]:
# also create versionless lookup
ens2entrez_core = {k.split(".")[0]: v for k, v in ens2entrez.items()}
ens2uniprot_core = {k.split(".")[0]: v for k, v in ens2uniprot.items()}

# now both will work
print("TP53 Entrez ID (with version):", ens2entrez.get("ENSG00000141510.12"))
print("TP53 Entrez ID (no version):", ens2entrez_core.get("ENSG00000141510"))


TP53 Entrez ID (with version): None
TP53 Entrez ID (no version): 7157


In [66]:
import math

ens2entrez  = {k: v for k, v in ens2entrez.items() if v is not None and not (isinstance(v, float) and math.isnan(v))}
ens2uniprot = {k: v for k, v in ens2uniprot.items() if v is not None and not (isinstance(v, float) and math.isnan(v))}

print(dict(list(ens2entrez.items())[:5]))
print(dict(list(ens2uniprot.items())[:5]))


{'ENSG00000278267.1': '102466751', 'ENSG00000284332.1': '100302278', 'ENSG00000237613.2': '645520', 'ENSG00000268020.3': '79504', 'ENSG00000240361.2': '403263'}
{'ENSG00000186092.6': 'Q8NH21', 'ENSG00000284733.1': 'Q6IEY1', 'ENSG00000284662.1': 'Q6IEY1', 'ENSG00000187634.12': 'Q96NU1', 'ENSG00000188976.11': 'Q9Y3T9'}


In [85]:
# filter out None/NaN/empty
ens2entrez  = {k: v for k, v in ens2entrez.items() if v is not None and str(v) != "nan"}
ens2uniprot = {k: v for k, v in ens2uniprot.items() if v not in [None, [], "nan"]}

# print first 5 entries
print(dict(list(ens2entrez.items())[:10]))
print(dict(list(ens2uniprot.items())[:10]))


{'ENSG00000278267.1': '102466751', 'ENSG00000284332.1': '100302278', 'ENSG00000237613.2': '645520', 'ENSG00000268020.3': '79504', 'ENSG00000240361.2': '403263', 'ENSG00000186092.6': '79501', 'ENSG00000233750.3': '100420257', 'ENSG00000222623.1': '124906683', 'ENSG00000273874.1': '102465909', 'ENSG00000228463.10': '728481'}
{'ENSG00000186092.6': 'Q8NH21', 'ENSG00000284733.1': 'Q6IEY1', 'ENSG00000284662.1': 'Q6IEY1', 'ENSG00000187634.12': 'Q96NU1', 'ENSG00000188976.11': 'Q9Y3T9', 'ENSG00000187961.14': 'Q6TDP4', 'ENSG00000187583.11': 'Q494U1', 'ENSG00000187642.9': 'Q5SV97', 'ENSG00000188290.11': 'Q9HCC6', 'ENSG00000187608.10': 'P05161'}


In [68]:
adata_ens

AnnData object with n_obs × n_vars = 12 × 67016
    obs: 'condition', 'tissue', 'replicate'
    var: 'ensembl_core', 'gene_symbol'
    layers: 'counts'

In [69]:
adata_ens.obs['condition']

sample
B_theta_AT_bio1           Bt
B_theta_AT_bio2           Bt
B_theta_AT_bio3           Bt
C_diff_AT_bio1            Cd
C_diff_AT_bio2            Cd
C_diff_AT_bio3            Cd
Co_colonized_AT_bio1      Co
Co_colonized_AT_bio2      Co
Co_colonized_AT_bio3      Co
mock_AT_bio1            Mock
mock_AT_bio2            Mock
mock_AT_bio3            Mock
Name: condition, dtype: category
Categories (4, object): ['Bt', 'Cd', 'Co', 'Mock']

In [70]:
adata_ens.obs['replicate']

sample
B_theta_AT_bio1         bio1
B_theta_AT_bio2         bio2
B_theta_AT_bio3         bio3
C_diff_AT_bio1          bio1
C_diff_AT_bio2          bio2
C_diff_AT_bio3          bio3
Co_colonized_AT_bio1    bio1
Co_colonized_AT_bio2    bio2
Co_colonized_AT_bio3    bio3
mock_AT_bio1            bio1
mock_AT_bio2            bio2
mock_AT_bio3            bio3
Name: replicate, dtype: category
Categories (3, object): ['bio1', 'bio2', 'bio3']

## save objects

In [81]:
from pathlib import Path
import pickle
import anndata as ad


In [82]:
# --- paths ---
BASE = Path("/storage/users/job37yv/Projects/Franziska_faber")
DATA_PROCESSED_DIR = BASE / "data" / "processed_data"
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

H5AD_OUT   = DATA_PROCESSED_DIR / "host_bulk_fractional_counts_ensembl.h5ad"
DICT_OUT   = DATA_PROCESSED_DIR / "ensembl_mappings.pkl"

# --- save AnnData with only Ensembl features ---
adata_ens.write_h5ad(H5AD_OUT)
print(f"[OK] Saved Ensembl-filtered AnnData → {H5AD_OUT}")

# --- save dictionaries ---
with open(DICT_OUT, "wb") as f:
    pickle.dump({"ens2entrez": ens2entrez, "ens2uniprot": ens2uniprot}, f)
print(f"[OK] Saved mapping dictionaries → {DICT_OUT}")


[OK] Saved Ensembl-filtered AnnData → /storage/users/job37yv/Projects/Franziska_faber/data/processed_data/host_bulk_fractional_counts_ensembl.h5ad
[OK] Saved mapping dictionaries → /storage/users/job37yv/Projects/Franziska_faber/data/processed_data/ensembl_mappings.pkl


In [83]:
adata_ens

AnnData object with n_obs × n_vars = 12 × 67016
    obs: 'condition', 'tissue', 'replicate'
    var: 'ensembl_core', 'gene_symbol'
    layers: 'counts'

In [84]:
adata_ens.var_names

Index(['ENSG00000223972.5', 'ENSG00000227232.5', 'ENSG00000278267.1',
       'ENSG00000243485.5', 'ENSG00000284332.1', 'ENSG00000237613.2',
       'ENSG00000268020.3', 'ENSG00000240361.2', 'ENSG00000186092.6',
       'ENSG00000238009.6',
       ...
       'ENSG00000273739.1', 'ENSG00000276700.1', 'ENSG00000276312.1',
       'ENSG00000275757.1', 'ENSG00000278573.1', 'ENSG00000276017.1',
       'ENSG00000278817.1', 'ENSG00000277196.4', 'ENSG00000278625.1',
       'ENSG00000277374.1'],
      dtype='object', length=67016)

## reload

In [72]:
import anndata as ad
import pickle

# load AnnData
adata_ens = ad.read_h5ad(H5AD_OUT)

# load dictionaries
with open(DICT_OUT, "rb") as f:
    maps = pickle.load(f)

ens2entrez = maps["ens2entrez"]
ens2uniprot = maps["ens2uniprot"]

print("TP53 Entrez ID:", ens2entrez.get("ENSG00000141510"))
print("TP53 UniProt:", ens2uniprot.get("ENSG00000141510"))


TP53 Entrez ID: None
TP53 UniProt: None


# Differential expression (Python-only DE (voom-style WLS; handles fractional counts)

## Calculate

In [74]:
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.nonparametric.smoothers_lowess import lowess
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# ---------- Inputs ----------
# Expect: adata_ens with layers["counts"], obs["condition"] in {Bt, Cd, Co, Mock}
BASE = Path("/storage/users/job37yv/Projects/Franziska_faber")
DE_DIR = BASE / "analysis" / "DE"
DE_DIR.mkdir(parents=True, exist_ok=True)

layer = "counts"
assert layer in adata_ens.layers, f"Missing layer '{layer}'. Available: {list(adata_ens.layers.keys())}"

# counts: genes x samples
counts = pd.DataFrame(
    adata_ens.layers[layer],
    index=adata_ens.obs_names,   # samples
    columns=adata_ens.var_names  # genes
).T

meta = adata_ens.obs.loc[counts.columns].copy()
cond = meta["condition"].astype("category")
if "Mock" not in cond.cat.categories:
    raise ValueError("Reference 'Mock' not in condition.")
# Put Mock first for intuitive contrasts
cond = cond.cat.reorder_categories(["Mock"] + [c for c in cond.cat.categories if c != "Mock"], ordered=True)
meta["condition"] = cond

print("[info] Samples per condition:")
print(meta["condition"].value_counts())

# ---------- Normalization: library-size CPM then log ----------
lib_sizes = counts.sum(axis=0).values  # per-sample library sizes
lib_sizes[lib_sizes == 0] = 1.0
cpm = counts.div(lib_sizes, axis=1) * 1e6

# Add a small offset to stabilize logs for low counts (as voom does via prior.count)
prior_count = 0.5
logCPM = np.log2(cpm + prior_count)

# ---------- Design matrix (no intercept: columns per level) ----------
X = pd.get_dummies(meta["condition"], drop_first=False)  # columns: Mock, Bt, Cd, Co
# We’ll compute contrasts manually below.
X_mat = X.values

# ---------- voom-like weights ----------
# Estimate mean–variance trend of logCPM:
# 1) mean expression per gene (across samples)
mu = logCPM.mean(axis=1).values  # (G,)
# 2) residual variance per gene across samples
#    Start with unweighted variance as a first pass
sigma2_gene = logCPM.var(axis=1, ddof=1).values  # (G,)

# LOWESS: predict variance as a smooth function of mean
# guard: remove NaNs/Infs
ok = np.isfinite(mu) & np.isfinite(sigma2_gene)
fit = lowess(sigma2_gene[ok], mu[ok], frac=0.3, return_sorted=True)
mu_grid, var_smooth = fit[:,0], fit[:,1]

# Map each gene's mu to a smoothed variance via nearest neighbor on the grid
# (simple & fast; spline would be nicer but overkill)
order = np.argsort(mu_grid)
mu_sorted = mu_grid[order]
var_sorted = np.maximum(var_smooth[order], 1e-6)
idx = np.searchsorted(mu_sorted, mu)
idx = np.clip(idx, 0, len(mu_sorted)-1)
var_hat_gene = var_sorted[idx]  # (G,)

# Turn gene-level variance into observation-level precision weights:
# simplest choice: w_gs = 1 / var_hat_gene for all samples s of gene g
weights = (1.0 / var_hat_gene)[:, None] * np.ones_like(logCPM.values)

# ---------- Fit per-gene weighted least squares ----------
Y = logCPM.values  # (G x N)

# Precompute (X' W X)^{-1} per gene efficiently:
# Because W is diagonal (weights per sample), we can scale X and y by sqrt(w)
sqrtW = np.sqrt(weights)  # (G x N)

coef = []
se = []
tval = []
df_resid = Y.shape[1] - X_mat.shape[1]

# Precompute X once per gene with weights
for g in range(Y.shape[0]):
    y = Y[g, :] * sqrtW[g, :]
    Xw = X_mat * sqrtW[g, :][:, None]
    XtX = Xw.T @ Xw
    try:
        XtX_inv = np.linalg.inv(XtX)
    except np.linalg.LinAlgError:
        # fallback: pseudo-inverse if singular
        XtX_inv = np.linalg.pinv(XtX)
    beta = XtX_inv @ (Xw.T @ y)
    # residuals
    yhat = X_mat @ beta
    resid = (Y[g, :] - yhat)
    # weighted residual variance (sigma^2)
    # Using weights: sum(w * e^2)/(N - p)
    sig2 = np.sum(weights[g, :] * resid**2) / max(df_resid, 1)
    # standard errors of coefficients
    covb = sig2 * XtX_inv
    se_g = np.sqrt(np.diag(covb))
    t_g = beta / se_g
    coef.append(beta)
    se.append(se_g)
    tval.append(t_g)

coef = np.vstack(coef)  # (G x P)
se   = np.vstack(se)    # (G x P)
tval = np.vstack(tval)  # (G x P)

# Columns of X (and coef) are in this order:
coef_cols = list(X.columns)  # ['Mock','Bt','Cd','Co']

# ---------- Define contrasts ----------
# Each contrast is a vector c over columns [Mock, Bt, Cd, Co]
def contrast_vec(name):
    m = {col:i for i,col in enumerate(coef_cols)}
    if name == "Bt_vs_Mock":
        c = np.zeros(len(coef_cols)); c[m["Bt"]]  = 1; c[m["Mock"]] = -1
    elif name == "Cd_vs_Mock":
        c = np.zeros(len(coef_cols)); c[m["Cd"]]  = 1; c[m["Mock"]] = -1
    elif name == "Co_vs_Mock":
        c = np.zeros(len(coef_cols)); c[m["Co"]]  = 1; c[m["Mock"]] = -1
    elif name == "Bt_vs_Cd":
        c = np.zeros(len(coef_cols)); c[m["Bt"]]  = 1; c[m["Cd"]]   = -1
    elif name == "Bt_vs_Co":
        c = np.zeros(len(coef_cols)); c[m["Bt"]]  = 1; c[m["Co"]]   = -1
    elif name == "Cd_vs_Co":
        c = np.zeros(len(coef_cols)); c[m["Cd"]]  = 1; c[m["Co"]]   = -1
    else:
        raise ValueError(name)
    return c

contrasts = ["Bt_vs_Mock","Cd_vs_Mock","Co_vs_Mock","Bt_vs_Cd","Bt_vs_Co","Cd_vs_Co"]

# For each contrast, compute:
#   logFC = c' * beta
#   SE(logFC) = sqrt(c' * Cov(beta) * c)
# where Cov(beta) ≈ average across genes? Better: use each gene's covb via sig2*XtX_inv computed above.
# We didn't keep per-gene covb but we can recompute SE from se & XtX_inv — to keep it lightweight,
# estimate SE via delta method using per-gene XtX_inv and sigma^2 already embedded in se.
# Since we don't store XtX_inv per gene, recompute quickly inside loop.

results = {}
gene_names = counts.index.to_numpy()

# Precompute per-gene XtX_inv and sigma^2 again (cheap)
XtX_inv_all = []
sigma2_all = []
for g in range(Y.shape[0]):
    y = Y[g, :] * sqrtW[g, :]
    Xw = X_mat * sqrtW[g, :][:, None]
    XtX = Xw.T @ Xw
    try:
        XtX_inv = np.linalg.inv(XtX)
    except np.linalg.LinAlgError:
        XtX_inv = np.linalg.pinv(XtX)
    yhat = X_mat @ (XtX_inv @ (Xw.T @ y))
    resid = (Y[g, :] - yhat)
    sig2 = np.sum(weights[g, :] * resid**2) / max(df_resid, 1)
    XtX_inv_all.append(XtX_inv)
    sigma2_all.append(sig2)
XtX_inv_all = np.array(XtX_inv_all)  # (G x P x P)
sigma2_all = np.array(sigma2_all)    # (G,)

for name in contrasts:
    c = contrast_vec(name)
    # contrast estimate per gene
    logFC = coef @ c
    # SE via sqrt( sigma^2 * c' (XtX)^-1 c )
    # vectorized:
    c_col = c.reshape(-1,1)
    tmp = XtX_inv_all @ c_col        # (G x P x 1)
    c_var = (c.reshape(1,1,-1) @ XtX_inv_all @ c_col).reshape(-1)  # (G,)
    se_contrast = np.sqrt(np.maximum(sigma2_all * c_var, 1e-12))
    t = logFC / se_contrast
    # two-sided p-values from t with df_resid
    from scipy.stats import t as student_t
    pval = 2 * (1 - student_t.cdf(np.abs(t), df=df_resid))
    padj = multipletests(pval, method="fdr_bh")[1]

    # AveExpr as average logCPM per gene
    ave_expr = logCPM.mean(axis=1).values

    df = pd.DataFrame({
        "gene": gene_names,
        "logFC": logFC,
        "AveExpr": ave_expr,
        "t": t,
        "pval": pval,
        "padj": padj
    })
    # insert gene_symbol if present
    if "gene_symbol" in adata_ens.var.columns:
        gene_map = pd.Series(adata_ens.var["gene_symbol"].astype(str).values,
                             index=adata_ens.var_names.astype(str))
        df.insert(1, "gene_symbol", gene_map.reindex(df["gene"]).values)

    out = DE_DIR / f"python_voomlite_{name}.csv"
    df.sort_values("pval", inplace=True)
    df.to_csv(out, index=False)
    results[name] = df
    print(f"[ok] Wrote {out}")

# (optional) quick summary
summary = []
for k, df in results.items():
    sig = df[df["padj"] < 0.05]
    sig_lfc1 = sig[sig["logFC"].abs() >= 1]
    summary.append({
        "contrast": k,
        "n_sig_FDR<0.05": len(sig),
        "n_sig_|logFC|>=1": len(sig_lfc1),
        "up_FDR<0.05": (sig["logFC"] > 0).sum(),
        "down_FDR<0.05": (sig["logFC"] < 0).sum()
    })
pd.DataFrame(summary).to_csv(DE_DIR / "python_voomlite_summary.csv", index=False)
print("[ok] Summary ->", DE_DIR / "python_voomlite_summary.csv")


[info] Samples per condition:
condition
Mock    3
Bt      3
Cd      3
Co      3
Name: count, dtype: int64


/home/job37yv/miniforge3/envs/scAnalysis/lib/python3.10/site-packages/statsmodels/nonparametric/smoothers_lowess.py:226: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),
/tmp/ipykernel_896322/4171479770.py:108: RuntimeWarning: divide by zero encountered in divide
  t_g = beta / se_g


[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Bt_vs_Mock.csv
[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Cd_vs_Mock.csv
[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Co_vs_Mock.csv
[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Bt_vs_Cd.csv
[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Bt_vs_Co.csv
[ok] Wrote /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_Cd_vs_Co.csv
[ok] Summary -> /storage/users/job37yv/Projects/Franziska_faber/analysis/DE/python_voomlite_summary.csv


## Visualize

In [80]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import pdist, squareform

# --- Paths & inputs ---
BASE = Path("/storage/users/job37yv/Projects/Franziska_faber")
ANALYSIS_DIR = BASE / "analysis"
DE_DIR  = ANALYSIS_DIR / "DE"
FIG_DIR = ANALYSIS_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

contrasts = ["Bt_vs_Mock","Cd_vs_Mock","Co_vs_Mock","Bt_vs_Cd","Bt_vs_Co","Cd_vs_Co"]
res = {c: pd.read_csv(DE_DIR / f"python_voomlite_{c}.csv") for c in contrasts}
for c in contrasts:
    res[c]["gene"] = res[c]["gene"].astype(str)

# Universe of tested genes (intersection across contrasts)
gene_universe = set.intersection(*[set(df["gene"]) for df in res.values()])

# --- Volcano plots ---
def volcano_plot(df, title, outpath, padj_thr=0.05, lfc_thr=1.0):
    x = df["logFC"].values
    y = -np.log10(np.clip(df["pval"].values, 1e-300, 1.0))
    plt.figure(figsize=(7,5))
    plt.scatter(x, y, s=6, alpha=0.4)
    mask = (df["padj"] < padj_thr) & (np.abs(df["logFC"]) >= lfc_thr)
    plt.scatter(df.loc[mask, "logFC"], -np.log10(np.clip(df.loc[mask, "pval"], 1e-300, 1.0)), s=6)
    plt.axvline(-lfc_thr, linestyle="--"); plt.axvline(lfc_thr, linestyle="--")
    plt.axhline(-np.log10(padj_thr), linestyle="--")
    plt.xlabel("log2 Fold Change"); plt.ylabel("-log10 p-value")
    plt.title(f"Volcano: {title}")
    plt.tight_layout(); plt.savefig(outpath, dpi=160); plt.close()

for c in contrasts:
    volcano_plot(res[c], c, FIG_DIR / f"volcano_{c}.png")

# --- logFC correlation heatmap across contrasts ---
genes_sorted = sorted(gene_universe)
M = np.vstack([res[c].set_index("gene").loc[genes_sorted, "logFC"].values for c in contrasts])
corr = np.corrcoef(M)
plt.figure(figsize=(6,5))
plt.imshow(corr, interpolation="nearest")
plt.xticks(range(len(contrasts)), contrasts, rotation=45, ha="right")
plt.yticks(range(len(contrasts)), contrasts)
plt.colorbar(label="Pearson r")
plt.title("logFC correlation across contrasts")
plt.tight_layout(); plt.savefig(FIG_DIR / "corr_logFC_all.png", dpi=160); plt.close()

# --- Clustered heatmap of top-union genes (rows & columns dendrograms) ---
# Recompute logCPM for plotting (same transform used in DE)
counts = pd.DataFrame(
    adata_ens.layers["counts"],
    index=adata_ens.obs_names,   # samples
    columns=adata_ens.var_names  # genes (Ensembl IDs, may include version)
).T
lib_sizes = counts.sum(axis=0).values
lib_sizes[lib_sizes == 0] = 1.0
cpm = counts.div(lib_sizes, axis=1) * 1e6
logCPM = np.log2(cpm + 0.5)

# Top union = top 50 per vs-Mock contrast
top_union = set()
for c in ["Bt_vs_Mock","Cd_vs_Mock","Co_vs_Mock"]:
    df = res[c].sort_values(["padj","pval","logFC"], ascending=[True, True, False])
    top_union.update(df["gene"].head(50).tolist())
sel_genes = [g for g in top_union if g in logCPM.index]

# --- robust Ensembl (±version) -> gene symbol mapping
def strip_ver(x: str) -> str:
    return re.sub(r"\.\d+$", "", x)

# Prefer 'gene_symbol', fall back to 'gene_symbols'
sym_col = "gene_symbol" if "gene_symbol" in adata_ens.var.columns else (
          "gene_symbols" if "gene_symbols" in adata_ens.var.columns else None)

if len(sel_genes) > 0:
    # Build a mapping keyed by *core* Ensembl ID (no version)
    if sym_col is not None:
        # If you already keep stripped IDs in var['ensembl_core'], use that; else strip var_names.
        if "ensembl_core" in adata_ens.var.columns:
            core_index = adata_ens.var["ensembl_core"].astype(str).values
        else:
            core_index = pd.Index(adata_ens.var_names.astype(str)).map(strip_ver).values
        ens_to_sym = pd.Series(
            adata_ens.var[sym_col].astype(str).values,
            index=core_index
        )
    else:
        ens_to_sym = None  # no symbols available

    sel_genes_core = [strip_ver(g) for g in sel_genes]
    row_labels_all = [ens_to_sym.get(core, core) if ens_to_sym is not None else core
                      for core in sel_genes_core]

    # Matrix & z-score per gene
    Z = logCPM.loc[sel_genes]
    Z = (Z - Z.mean(axis=1).values[:, None]) / (Z.std(axis=1).values[:, None] + 1e-9)

    # Cluster rows (euclidean, ward) and columns (correlation distance, average)
    row_link  = linkage(pdist(Z.values, metric="euclidean"), method="ward")
    row_order = leaves_list(row_link)

    Zt = Z.values.T
    C  = np.corrcoef(Zt)
    D  = np.clip(1 - C, 0, 2)
    col_link  = linkage(squareform(D, checks=False), method="average")
    col_order = leaves_list(col_link)

    # Reorder matrix & labels
    Z_ord      = Z.values[row_order][:, col_order]
    row_labels = [row_labels_all[i] for i in row_order]
    col_labels = logCPM.columns[col_order]

    # Draw dendrogram heatmap
    fig = plt.figure(figsize=(max(8, 0.4*len(col_labels)+4),
                              max(8, 0.18*len(row_labels)+4)))
    top = 0.93; left = 0.22; dendro_h = 0.15; dendro_w = 0.15; heat_w = 0.7; heat_h = 0.7

    ax_col = fig.add_axes([left, top - dendro_h, heat_w, dendro_h])
    dendrogram(col_link, ax=ax_col, no_labels=True, color_threshold=None)
    ax_col.set_xticks([]); ax_col.set_yticks([])

    ax_row = fig.add_axes([left - dendro_w, top - dendro_h - heat_h, dendro_w, heat_h])
    dendrogram(row_link, ax=ax_row, orientation="right", no_labels=True, color_threshold=None)
    ax_row.set_xticks([]); ax_row.set_yticks([])

    ax_heat = fig.add_axes([left, top - dendro_h - heat_h, heat_w, heat_h])
    im = ax_heat.imshow(Z_ord, aspect="auto", interpolation="nearest")
    ax_heat.set_xticks(range(len(col_labels)))
    ax_heat.set_xticklabels(col_labels, rotation=90, fontsize=8)
    ax_heat.set_yticks(range(len(row_labels)))
    ax_heat.set_yticklabels(row_labels, fontsize=7)  # always gene symbols when available
    ax_heat.set_title("Top-union DE genes (z-scored logCPM)")

    cax = fig.add_axes([left + heat_w + 0.01, top - dendro_h - heat_h, 0.02, heat_h])
    cb = plt.colorbar(im, cax=cax); cb.set_label("Z-score")

    plt.savefig(FIG_DIR / "heatmap_top_union_clustered.png", dpi=160, bbox_inches="tight")
    plt.close()

print("[OK] Wrote volcano plots and correlation heatmap to", FIG_DIR)
print("[OK] Wrote clustered top-union heatmap with dendrograms to", FIG_DIR / "heatmap_top_union_clustered.png")


[OK] Wrote volcano plots and correlation heatmap to /storage/users/job37yv/Projects/Franziska_faber/analysis/figures
[OK] Wrote clustered top-union heatmap with dendrograms to /storage/users/job37yv/Projects/Franziska_faber/analysis/figures/heatmap_top_union_clustered.png
